Read in all the packages

In [1]:
import os
import ee
import numpy as np
from geeml.extract import extractor
import pandas as pd
import random

# import geemap
# Authenticate GEE
# ee.Authenticate()
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/explore/nobackup/people/spotter5/cnn_mapping/gee-serdp-upload-7cd81da3dc69.json"

service_account = 'gee-serdp-upload@appspot.gserviceaccount.com'
credentials = ee.ServiceAccountCredentials(service_account, "/explore/nobackup/people/spotter5/cnn_mapping/gee-serdp-upload-7cd81da3dc69.json")
ee.Initialize(credentials)
# Initialize GEE with high-volume end-point
# ee.Initialize(opt_url='https://earthengine-highvolume.googleapis.com')
ee.Initialize()


In [2]:
import geemap
from google.cloud import storage
from google.cloud import client

Read in assets

In [3]:
sent_2A = ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED") #sentinel 2
s2Clouds = ee.ImageCollection('COPERNICUS/S2_CLOUD_PROBABILITY') #cloud masking for sentinel
lfdb = ee.FeatureCollection("users/spotter/fire_cnn/raw/ea_grid") 

#need to add ids to annas polygons
# Create a sequence of numbers from 1 to the number of features
# indexes = lfdb.aggregate_array('system:index')
# ids = list(range(1, indexes.size().getInfo() + 1))
# idByIndex = ee.Dictionary.fromLists(indexes, ids)

# # Map over the collection and set the 'ID' property
# lfdb = lfdb.map(lambda feature: feature.set('ID', idByIndex.get(feature.get('system:index'))))



Mask clouds Sentinel

In [4]:

#probability of clouds
MAX_CLOUD_PROBABILITY = 50

def sent_maskcloud(image):
    
    
    image = image.select(['B2', 'B3', 'B4', 'B8', 'B11', 'B12'], ['SR_B1', 'SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B7'])# rename bands to match landsat
  
    image =  image.toShort()
    
    clouds = ee.Image(image.get('cloud_mask')).select('probability')
    
    isNotCloud = clouds.lt(MAX_CLOUD_PROBABILITY)
    
    image = image.updateMask(isNotCloud)

    #reproject 30m but remember b1, b2 and b3 are 10 and the rest are 20
    image1 = image.select(['SR_B1', 'SR_B2', 'SR_B3', 'SR_B4'])
    image2 = image.select(['SR_B5', 'SR_B7'])

    
    image1 = image1.reproject(
    crs = image1.projection().crs(),
    scale = 30) #resample for landsat
    
    
    image2 = image2.reproject(
    crs = image2.projection().crs(),
    scale = 30) #resample for landsat
    
    image = image1.addBands(image2)
    
    return image 

#Join S2 SR with cloud probability dataset to add cloud mask.
s2SrWithCloudMask = ee.Join.saveFirst('cloud_mask').apply(
    
  primary=sent_2A,
  secondary=s2Clouds,
  condition=ee.Filter.equals(leftField='system:index', rightField='system:index'))

#apply cloud masking
sent_2A = ee.ImageCollection(s2SrWithCloudMask).map(sent_maskcloud)


Mask clouds


In [5]:
def mask_s2clouds(image):
    qa = image.select('QA60')

    # Bits 10 and 11 are clouds and cirrus, respectively.
    cloud_bit_mask = 1 << 10
    cirrus_bit_mask = 1 << 11

    # Both flags should be set to zero, indicating clear conditions.
    mask = qa.bitwiseAnd(cloud_bit_mask).eq(0) \
        .And(qa.bitwiseAnd(cirrus_bit_mask).eq(0))

    return image.updateMask(mask).divide(10000) #dividing is the scale factor application

Coefficients from Logan Berner to apply correction factors

Authenticate my google bucket, not sure if this is needed anymore

In [6]:
os.environ["GCLOUD_PROJECT"] = "gee-serdp-upload"

os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/explore/nobackup/people/spotter5/cnn_mapping/gee-serdp-upload-7cd81da3dc69.json"
storage_client = storage.Client.from_service_account_json("/explore/nobackup/people/spotter5/cnn_mapping/gee-serdp-upload-7cd81da3dc69.json")

os.environ["GCLOUD_PROJECT"] = "gee-serdp-upload"
storage_client = storage.Client()
# bucket_name = 'smp-scratch/mtbs_1985'
bucket_name = 'smp-scratch'

bucket = storage_client.bucket(bucket_name)

There are going to be issues later with different band types, so cast them all to float

In [7]:
def to_float(image):

    b1 = image.select('SR_B1').cast({'SR_B1':'float'}) #0
    b2 = image.select('SR_B2').cast({'SR_B2':'float'}) #1
    b3 = image.select('SR_B3').cast({'SR_B3':'float'}) #2
    b4 = image.select('SR_B4').cast({'SR_B4':'float'}) #3
    b5 = image.select('SR_B5').cast({'SR_B5':'float'}) #4
    b6 = image.select('SR_B7').cast({'SR_B7':'float'}) #5

    image = b1.addBands(b2).addBands(b3).addBands(b4).addBands(b5).addBands(b6)

    return image

Function to get the HLS post fire data for a specific time frame.  ONly need to specify post fire dates here.  I will download a variety using different months to test.  Note HLS is only 2017-present.  It seems really only 2019 though.

In [8]:
def get_imagery(start, end, geometry):
    
    #hls image collection, filter date, geometry, cloud masking, take median
    hls = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED').filterDate(start, end).filterBounds(geometry).filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE',20)).map(mask_s2clouds).median()
    
    #resample blue green red to 20m
    bgr = hls.select(['B2', 'B3', 'B4'])
    
    #select nir, swir1, swir2, already at 20m
    ns = hls.select(['B8', 'B11', 'B12'])
    
    bgr= bgr.reproject(
        crs = bgr.projection().crs(),
        scale = 20) 
    
    
    image = bgr.addBands(ns)
    
    #return the image
    return image
    
    
    

In [11]:
pre_months = ['-04-01', '-05-01', '-06-01', '-07-01', '-08-01', '-09-01']
end_months = ['-05-01', '-06-01', '-07-01', '-08-01', '-09-01', '-10-01']

all_months = dict(zip(pre_months, end_months))

#these ids have a mix of grassland (early season) and siberia (late season) burns.
all_ids = [186, 131, 175, 189, 72, 163]
#loop through each fire polygon

year = 2020

for i in all_ids:
    
    sub_shape = lfdb.filter(ee.Filter.eq('Id', i))
        
        
     #loop through all months 
    for m1, m2 in all_months.items():
        
        m1_name = m1.replace('-', '_')
        m2_name = m2.replace('-', '_')
        #name of output file
        fname = f"{year}{m1_name}{m2_name}_hls_{i}.tif"

        #check if file exists on my bucket, if it does skip
        stats = storage.Blob(bucket=bucket, name=fname).exists(storage_client)

        if stats == False:

            #get imagary dates

            start = str(year) + m1
            end = str(year) + m2



            #apply the function to get the pre_fire image and post_fire image
            this_image =  get_imagery(start, end, sub_shape)
            

            #start download
            print(f"Downloading {fname}")


            #export image to my cloud storage
            task = ee.batch.Export.image.toCloudStorage(
                                    image = this_image.multiply(1000).toShort(),
                                    region=sub_shape.geometry(), 
                                    description=fname,
                                    scale=20,
                                    crs='EPSG:3413',
                                    maxPixels=1e13,
                                    bucket = 'smp-scratch')

            task.start()

        
        
        
        
        




In [10]:
# Use reduceRegion to calculate the minimum and maximum values in the specified ROI
stats = raw_bands.select('NBR_p85').reduceRegion(
    reducer=ee.Reducer.minMax(),
    geometry=final_buffer,
    scale=30,  # Set the scale according to the resolution of the image
    maxPixels=1e9  # Adjust maxPixels as needed
)

# Get the minimum and maximum values
min_value = stats.getNumber('NBR_p85_min')
max_value = stats.getNumber('NBR_p85_max')

# Print the results
print('Minimum Value:', min_value.getInfo())
print('Maximum Value:', max_value.getInfo())

NameError: name 'raw_bands' is not defined